# 引言

最简单的神经元：房价预测

<img src="images/最简单的神经元.png" width="300"/>

为什么用神经网络

<img src="images/为什么用神经网络.png" width="300"/>    

数据集越大，对越大的神经网络的性能越好。   
对于比较小的数据集，各种模型的性能差不多

# 第一部分 ： 二分类神经网络

### （一）样品的数据结构：

一张已经被标记为“xxx”的图 被称作样品

<img src="images/三色图.png" alt="image1" width="300"/>


图像由形状$（长度，高度，深度=3）$的3D阵列表示。然而，当您读取图像作为算法的输入时，您可以将其转换为形状为$（长度*高度*3，1）$的向量。    
换句话说，可以将三维阵列“展开”或重塑为一维矢量。
也就是把3个图层的数据连起来排成一列，成为一个  $x$    
具体方法如下：

In [ ]:
def image2vector(image):

    a=image.shape[0]
    b=image.shape[1]
    c=image.shape[2]
    v=image.reshape(a*b*c,1)

    return v

处理好图像后，得到向量x作为输入变量     

**输入**  
$x$: 表示一个 $n_x$ 维度特征，为输入特征，维度为 $(n_x, 1)$。


---

**输出**  
$y$: 表示输出结果，取值为 $(0,1)$;  
$(x^{(i)}, y^{(i)})$: 表示第 $i$ 组数据，可能是训练数据，也可能是测试数据，此处暂译为训练数据;

---

$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $


### （2）逻辑回归（Logistic Regression）

对于二元分类问题来说，给定一个输入特征向量 $x$，它可能属于一类或另一类，模型的任务就是找出其属于哪一类。   
我们的模型在计算过程中，需要将输入特征 $x$ 转换为输出估计值 $\hat{y}$。    
比如对于猫图而言，如果“是猫图”的 $y$ 表示为 1， “不是猫图”的 $y$ 表示为 0   
那么  $\hat{y}$  需要在（0,1）之内，表示“是猫图”的可能性



在开始之前，我们先介绍一下用sigmoid函数来处理向量x

$$ \text{For } x \in \mathbb{R}^n \text{,     } sigmoid(x) = sigmoid\begin{pmatrix}
    x_1  \\
    x_2  \\
    ...  \\
    x_n  \\
\end{pmatrix} = \begin{pmatrix}
    \frac{1}{1+e^{-x_1}}  \\
    \frac{1}{1+e^{-x_2}}  \\
    ...  \\
    \frac{1}{1+e^{-x_n}}  \\
\end{pmatrix}\tag{1} $$

sigmoid函数的导数为：
$$sigmoid\_derivative(x) = \sigma'(x) = \sigma(x) (1 - \sigma(x))\tag{2}$$

In [ ]:
import numpy as np 

def sigmoid(x):
    s=1/(1+np.exp(-x))
    return s
def sigmoid_derivative(x):
    ds=sigmoid(x)*(1-sigmoid(x))    
    return ds

接下来就是预测的模型

$$
\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b), \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}
$$

Given $\{(x^{(1)}, y^{(1)}), \dots, (x^{(m)}, y^{(m)})\}$, 而我们希望 $\hat{y}^{(i)} \approx y^{(i)}$.


如何来衡量模型的准确性呢？  
需要用到 “损失函数”（loss function）：

$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$

当 y=1 时，只有 $\hat{y}$ 尽可能大（收范围限制趋近于1），损失函数L才会小   
当 y=0 时，只有 $\hat{y}$ 尽可能小（收范围限制趋近于0），损失函数L才会小

然而这只是对于一个样本的衡量方法，对于一个样本集而言，需要进行累加，这称为 成本函数（Cost Function）

$$
J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})
$$
    

即
$$
J(w, b)= \frac{1}{m} \sum_{i=1}^m \left(-y^{(i)} \log(\hat{y}^{(i)}) - (1 - y^{(i)}) \log(1 - \hat{y}^{(i)})\right)
$$


那么现在优化的方向就已经相当明确了，我们要不断修改w和b来让成本函数尽量小   
而事实上，使用了sigmoid 函数，这个成本函数是有最低点的 



<img src='images/成本函数3D.png'><img>

所以直接使用 **梯度下降法**：

假设b不变时：
$$
w = w - a \frac{dJ(w)}{dw}
$$



其中      
𝑎 ：学习率（ learning rate）    
$a \frac{dJ(w)}{dw}$ : 步长 (step），即向下走一步的长度

注意：   
对于 $\frac{dJ(w)}{dw}$  我们一般简写做 $dw$

---

拓展到两个参数就是：
$$
w := w - a \frac{\partial J(w, b)}{\partial w}, \quad b := b - a \frac{\partial J(w, b)}{\partial b}
$$


    

### （3）逻辑回归中的梯度下降（Logistic Regression Gradient Descent）

我们在前面已经了解了逻辑回归的训练过程    

那么怎么计算 $\frac{\partial J(w, b)}{\partial w}$  和  $\frac{\partial J(w, b)}{\partial b}$   呢？

假设样本有2个特征 $x_1,x_2$，那么 z 的表达式应该修改为：

$$
z=w_1x_1+w_2x_2+b
$$


回忆一下：   
$\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b) = a, \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}$  

其损失函数为：   
$L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})$    

对于单个样本而言，代价函数 $J(w,b) $ 就是损失函数：   
$L(a,y) = -ylog(a)-(1-y)log(1-a)$    

其中𝑎是逻辑回归的输出， 𝑦是样本的标签值

求导可得：   
$$ \frac{dL(a,y)}{da} = -\frac{y}{a}+\frac{1-y}{1-a} $$




同时：
$$
\begin{align*}
a &= \sigma(z) = \frac{1}{1+e^{-z}} \\
\implies 1 + e^{-z} &= \frac{1}{a} \\
\implies -e^{-z} \, dz &= -\frac{1}{a^2} \, da \\
\implies \frac{da}{dz} &= a^2 e^{-z} \\
&= a^2 \left( \frac{1}{a} - 1 \right) \\
&= a(1-a)
\end{align*}
$$


因为：   
$$ 
\frac{dL}{dz}  =\frac{dL}{da} \frac{da}{dz}     \\
$$
$$
\implies \frac{dL}{dz} =(-\frac{y}{a}+\frac{1-y}{1-a}) *  a(1-a)
$$
$$
\implies \frac{dL}{dz}= a-y
$$

同时，我们一般直接用 $dz$ 来表示 $\frac{dL}{dz}$

因为：
$$ 
\frac{dL}{dw}  =\frac{dL}{dz} \frac{dz}{dw}     \\
$$
$$
\implies \frac{dL}{dw} =(a-y)(x)
$$

所以：   
$\frac{dL}{dw_1} =(a-y)(x_1)$    

$\frac{dL}{dw_2} =(a-y)(x_2)$


因为：
$$ 
\frac{dL}{db}  =\frac{dL}{dz} \frac{dz}{db}     \\
$$
$$
\implies \frac{dL}{db} =a-y
$$

综上，对于单个样本的梯度下降算法：    
$$
w_1=w_1-adw_1  \\
w_2=w_2-adw_2   \\
b=b-adb
$$
其中
$$
dw_1 =(a-y)(x_1) \\
dw_2 =(a-y)(x_2)\\
db=a-y\\

$$

上述结论拓展到m个样本应该如何呢？     
我们知道：
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})  $     
即：  
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(a^{(i)}, y^{(i)})$    
其中 $a^{(i)}$ 是第 i 个样本的预测输出，$y^{(i)}$是第 i 个样本的实际标签。








对权重 \(w_1\) 的梯度：   
$$
\frac{\partial J}{\partial w_1} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_1^{(i)}
$$

对权重 \(w_2\) 的梯度：
$$
\frac{\partial J}{\partial w_2} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_2^{(i)}
$$

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$

梯度下降更新规则
使用上述计算出的平均梯度来更新参数：
$$
w_1 = w_1 - \alpha \frac{\partial J}{\partial w_1} \\
w_2 = w_2 - \alpha \frac{\partial J}{\partial w_2} \\
b = b - \alpha \frac{\partial J}{\partial b}
$$

其中，$\alpha$ 是学习率，用于控制更新步骤的大小。

### （四）向量化（Vectorization）

在上面的计算方法中,每进行一次梯度下降时，都需要使用for循环来遍历每一个样本    
事实上这样的效率非常低，所以我们学习使用 **向量化** 来解决这个问题

所谓向量化，就是对于一串数据，比如数组或者矩阵，本来采用for循环进行一个个的运算，    
但是现在使用numpy库的自带功能，去掉显式的for循环     


In [12]:
import numpy as np
import time

data = np.random.rand(1000000)  # 假设有一个大型数组

# 向量化计算平方根和平均值
tik = time.time()
sqrt_data_vectorized = np.sqrt(data)
tok = time.time()
time1 = tok - tik
print('time:', time1)
print(sqrt_data_vectorized[:5])  # 打印前5个元素作为示例


# 普通计算方法
tik = time.time()
sqrt_list = []  # 使用列表收集平方根结果
for i in data:
    sqrt_list.append(i**0.5)  # 向列表追加平方根
sqrt_data = np.array(sqrt_list)  # 将列表转换为NumPy数组
tok = time.time()
time2 = tok - tik    
print('time:', time2)
print(sqrt_data[:5])  # 打印前5个元素作为示例


Vectorized computation time: 0.0019941329956054688
[0.88992777 0.75336519 0.87875718 0.93422771 0.88558512]
Loop computation time: 0.12228894233703613
[0.88992777 0.75336519 0.87875718 0.93422771 0.88558512]


上面的结果可以看到相差了100多倍    
注意：  
所谓向量化，不一定要调用numpy的计算函数比如 np.mean() np.sqrt()等，而是使用了np.array 的数据格式   
比如对于一个 np.array 进行 **2 平方操作，也是向量化的

根据我们已经学会的算法，可以知道现在的计算过程如下：   

初始化 $J=0$, $dw_1=0$, $dw_2=0$, $db=0$。代码流程如下：

```python
J = 0; dw1 = 0; dw2 = 0; db = 0
for i in range(1, m+1):  # 假设m是样本数量
    z_i = w * x[i] + b  # 这里假设x[i]是一个包含x1和x2的向量
    a_i = sigmoid(z_i)
    J += -[y[i] * log(a_i) + (1 - y[i]) * log(1 - a_i)]
    dz_i = a_i - y[i]
    dw1 += x[i][0] * dz_i  # 假设x[i][0]是特征x1
    dw2 += x[i][1] * dz_i  # 假设x[i][1]是特征x2
    db += dz_i

# 外部循环结束后，计算平均值
J /= m
dw1 /= m
dw2 /= m
db /= m

# 更新参数
w1 = w1 - alpha * dw1
w2 = w2 - alpha * dw2
b = b - alpha * db


上面这段代码中实现了一次梯度下降，也就是一次训练，但是使用了两个循环     
第一个循环是for循环遍历每一个样本     
第二个循环是对特征值进行循环。在这例子我们有 2 个特征值。如果你有超过两个特征时，需要循环 𝑑𝑤1 、 𝑑𝑤2 、 𝑑𝑤3 等等。



我们先来看**第二个循环的向量化**      
不用初始化 𝑑𝑤1 𝑑𝑤2 都等于 0，而是定义 𝑑𝑤 为一个向量，设置 $ dw=np.zeros(n_x,1)  $  定义一个x行的一位向量

<img src="images/第一层向量化.png">

再来看 **第二个循环的向量化**     
回忆一下最开始讲到的输入集


$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $



所以先计算 $z_1,z_2,z_3, \dots, z_n$  ,把他们都放到一个  $1 \times m$ 的行向量中    
你可以发现他可以表达为  $w^T$  (w的转置) $\times X+[b,b,\dots,b]$    
$[b,b,\dots,b]$ 是一个 $1 \times m$ 的行向量

所以计算的最终得到的$Z$是一个 $1 \times m$ 的向量，$Z = \begin{bmatrix} z^{(1)} & z^{(2)} & \ldots & z^{(m)} \end{bmatrix}$，计算方式为 $z = w^T X + \mathbf{b}$，其中 $\mathbf{b} = \begin{bmatrix} b & b & \ldots & b \end{bmatrix}$   

$$
Z = \begin{bmatrix}
w^T x^{(1)} + b \\
w^T x^{(2)} + b \\
\vdots \\
w^T x^{(m)} + b
\end{bmatrix}
$$

其中，
- $w^T x^{(1)} + b$ 是向量 $Z$ 的第一个元素，
- $w^T x^{(2)} + b$ 是第二个元素，
- 以此类推，直到 $w^T x^{(m)} + b$ 是第 $m$ 个元素。



In [ ]:
Z=np.dot(w.T, X)+b
# 其中b通过广播机制自动被拓展成一个 $1 \times m$ 的行向量

加下来要使用向量Z计算出向量Y    
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

然后就可以计算 $ dZ =A-Y=[a^{(1)}-y^{(1)},a^{(2)}-y^{(2)},\dots, a^{(n)}-y^{(n)}   ]  $

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$
所以 $ db=\frac{1}{m}*\sum_{i=1}^m (dz^{(i)}) $   


In [ ]:
db=(1/m)*np.sum(dZ)

对权重 \(w\) 的梯度：
$$
\frac{\partial J}{\partial w} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x^{(i)}
$$

所以  $ db=\frac{1}{m}*X*dz^T $   

### (五)总结

这是没有使用向量化的计算过程

<img src='images/原始计算过程.png'>

这是使用向量化之后的计算过程   


<img src='images/两层向量化.png'>


### (六)注意事项

这里要讲一个重要且常见的bug——一维数组

In [1]:
import numpy as np

a=np.random.rand(5)
print(a.shape)  #(5,)
# 此时a就是一个一维数组，既不是行向量也不是列向量

(5,)


这种结构很容易出现意想不到的bug，所以是要坚决摈弃的，一定要修改成(5,1)

In [4]:
import numpy as np

a=np.random.rand(5,1)
print(a.shape)  #(5,1)

(5, 1)


# 第二部分：浅层神经网络（Shallow neural networks）